# Quantum Circuits
Quantum computers can only use a specific set of gates (universal gate set). Given the entanglers and their amplitudes found in Step 3, one can find corresponding representation of these operators in terms of elementary gates using the following procedure.

In [1]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']= 'True'

import tequila as tq
from utility import *

First, we set up the Hamiltonian in Tequila's format and the unitary gates obtained in Step 3. 

In [2]:
H = tq.QubitHamiltonian.from_openfermion(get_qubit_hamiltonian('h4', 4, 'sto-3g', qubit_transf='jw'))

a = tq.Variable("tau_0")
U = construct_QMF_ansatz(4)
U += tq.gates.ExpPauli(paulistring=tq.PauliString.from_string("X(0)Y(1)X(2)X(3)"), angle=a)
print(U)

circuit: 
Rx(target=(0,), parameter=beta_0)
Rz(target=(0,), parameter=gamma_0)
Rx(target=(1,), parameter=beta_1)
Rz(target=(1,), parameter=gamma_1)
Rx(target=(2,), parameter=beta_2)
Rz(target=(2,), parameter=gamma_2)
Rx(target=(3,), parameter=beta_3)
Rz(target=(3,), parameter=gamma_3)
Exp-Pauli(target=(0, 1, 2, 3), control=(), parameter=tau_0, paulistring=X(0)Y(1)X(2)X(3))



One can check the expectation value to see it is near the ground state energy.

In [3]:
E = tq.ExpectationValue(H=H, U=U)
vars = {'beta_1': 3.141592624143881, 'beta_0': 3.141592624143881, 'tau_0': 1.1331410014096885, 'gamma_1': 0.0, 'beta_3': 0.0, 'gamma_3': 0.0, 'gamma_2': 0.0, 'gamma_0': 0.0, 'beta_2': 0.0} # values obtained from step 3
print(tq.simulate(E, variables=vars))

-0.807560897041014


One can run the same experiment on a real quantum computer through IBM Quantum Experience (ibmq). After activating your account here (https://quantum-computing.ibm.com/login), copy the API token and execute the commented block below. 

In [4]:
from qiskit import IBMQ
IBMQ.save_account('d5ff44567a34cecfd6b02abd67c9e218757cb7c8306966ba248ea657725900a4ea8feac54dabc8356f89d3ce3e28635d7903f4e14dffd0980561f8bf8a8625c3')

configrc.store_credentials:WARNING:2022-07-17 14:26:27,855: Credentials already present. Set overwrite=True to overwrite.


## Using Quantum System

In [5]:
provider = IBMQ.load_account()

In [ ]:
tq.simulate(E, variables=vars, samples=100, backend="qiskit", device='ibmq_manila')

The following code block prints the circuit.

In [ ]:
circ = tq.circuit.compiler.compile_exponential_pauli_gate(U)
tq.draw(circ, backend="qiskit")

In [ ]:
tq.simulate(E, variables=vars, samples=100, backend="qiskit", device='ibmq_belem')


In [ ]:
tq.simulate(E, variables=vars, samples=1000, backend="qiskit", device='ibmq_belem')


In [ ]:
tq.simulate(E, variables=vars, samples=5000, backend="qiskit", device='ibmq_belem')

In [ ]:
tq.simulate(E, variables=vars, samples=8192, backend="qiskit", device='ibmq_belem')


In [ ]:
Ebins = np.zeros(20)
for bin in range(len(Ebins)):
    Ebins[bin] = tq.simulate(E, variables=vars, samples=1000, backend="qiskit", device='ibmq_belem')
print(Ebins)
print('Average Energy:',np.average(Ebins))
print('Stdev: ',np.std(Ebins))



In [ ]:
import matplotlib.pyplot as plt

samples = [100,1000,5000,8192]
Evals = [-0.6782390850939107, -0.7031310116153874, -0.7047891180121715, -0.6854283267691902]
plt.plot(samples,Evals,'or',label='IBM BELEM')
plt.hlines(-0.9486411121761622,0,8192,label='EXACT')

plt.title('H4 Energy in IBMQ_BELEM, STO-3G')
plt.xlabel('Samples',fontsize=20)
plt.ylabel(r'Energy ($E_h$)',fontsize=14)
plt.legend(loc='center right')


In [ ]:
samples = [100,1000,5000,8192]
Evals = [-0.6782390850939107, -0.7031310116153874, -0.7047891180121715, -0.6854283267691902]
plt.plot(range(len(Ebins)),Ebins,'or',label='IBM BELEM')
plt.hlines(-0.9486411121761622,0,19,label='EXACT',colors='black')
plt.hlines(np.average(Ebins),0,19,label='IBM BELEM AVG')
plt.hlines(np.average(Ebins)+np.std(Ebins),0,19,label=r'IBM BELEM +$\sigma$',linestyles='dashed')
plt.hlines(np.average(Ebins)-np.std(Ebins),0,19,label=r'IBM BELEM -$\sigma$',linestyles='dashed')

plt.title('H4 Energy in IBMQ_BELEM, STO-3G')
plt.xlabel('Samples',fontsize=20)
plt.ylabel(r'Energy ($E_h$)',fontsize=14)
plt.xticks(range(len(Ebins)))
plt.legend(loc='center right')

In [ ]:
circ = tq.circuit.compiler.compile_exponential_pauli_gate(U)
tq.draw(circ, backend="qiskit")